In [7]:
import requests, os
from bs4 import BeautifulSoup
from src.downloader import download

def get_html(url):
    """Get raw HTML from a URL."""
    headers = {
        'Access-Control-Allow-Origin': '*',
        'Access-Control-Allow-Methods': 'GET',
        'Access-Control-Allow-Headers': 'Content-Type',
        'Access-Control-Max-Age': '3600',
        'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:52.0) Gecko/20100101 Firefox/52.0'
    }
    req = requests.get(url, headers=headers)
    return req.text
    

def get_image(html):
    """Scrape share image."""
    image = []
    if html.find("meta", property="image"):
        image = html.find("meta", property="image").get('content')
    elif html.find("meta", property="og:image"):
        image = html.find("meta", property="og:image").get('content')
    elif html.find("meta", property="twitter:image"):
        image = html.find("meta", property="twitter:image").get('content')
    elif html.find("img", src=True):
        for img in html.find_all("img"):
            image.append(img.get('src'))
    return image


def check_word(word):
    def check(url):
        return (word in url)
    return check

In [14]:
keyword = "gadget electronique"
page = "2"
url = "https://www.amazon.fr/s?k={}&page={}&__mk_fr_FR=%C3%85M%C3%85%C5%BD%C3%95%C3%91&ref=sr_pg_{}".format(keyword, page, page)

soup = BeautifulSoup(get_html(url))
# print(soup.prettify())
for image_url in list(filter(check_word("_AC_UL320_"), get_image(soup))):
    download(image_url, "imgs")

SSLError: HTTPSConnectionPool(host='m.media-amazon.com', port=443): Max retries exceeded with url: /images/I/61YC5v2tk7L._AC_UL320_.jpg (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: self signed certificate in certificate chain (_ssl.c:1076)')))

In [8]:
import os, requests, time, shutil, glob, bz2, imgkit, re
import numpy as np
from PIL import Image
from multiprocessing.dummy import Pool
from src.azurekey import *

headers = {
    'Access-Control-Allow-Origin': '*',
    'Access-Control-Allow-Methods': 'GET',
    'Access-Control-Allow-Headers': 'Content-Type',
    'Access-Control-Max-Age': '3600',
    'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:52.0) Gecko/20100101 Firefox/52.0'
}

def search(search_term, azureKey, offset=0):
    # print('searching using bing: "'+search_term+'"')
    search_url = "https://api.bing.microsoft.com/v7.0/search"
    subscription_key = azureKey
    assert subscription_key
    headers = {"Ocp-Apim-Subscription-Key" : subscription_key}
    params  = {"q": search_term, "responseFilter": "Webpages","count":50, "offset":offset, "safeSearch":"Strict"}
    response = requests.get(search_url, headers=headers, params=params)
    response.raise_for_status()
    search_results = response.json()
    return [v["url"] for v in search_results["webPages"]["value"]]

def clean(html):
    html_temp = re.sub('href=".*?"', 'href=""', html)
    html_temp = re.sub('src=".*?"', 'src=""', html_temp)
    html_temp = re.sub('<script.*?/script>', '', html_temp)
    return html_temp

def save_html2jpg(url, dir_name):
    if not os.path.exists(dir_name):
        os.mkdir(dir_name)
    req = requests.get(url, headers=headers)
    imgkit.from_string(clean(req.text), os.path.join(dir_name, str(hash(url))+".jpg"))


def download(links_in, dir_name, n=None, log=set()):
    links = links_in[:n]
    # print('search results',len(links)
    if not os.path.exists(dir_name):
        os.mkdir(dir_name)

    def fetch(url, log=log):
        r=requests.get(url[0], stream=True)
        r.raw.decode_content = True # handle spurious Content-Encoding
        img = Image.open(r.raw)
        bits = str(imagehash.average_hash(img))
        if bits not in log:
            log |= set([bits])
        if len(log) % 100 == 0:
            print("{} images saved.".format(len(log)))
        img_file = dir_name+'/commercial_product_'+bits+".jpg"
        img.save(img_file)
        

    modLinks=[[links[i],i+1] for i in range(len(links))]
    Pool(10).map(fetch, modLinks)

    return log

In [16]:
search_results = search("electronics", azureKey, offset=0)

In [14]:
with open("webpages/test4.html", "w") as f:
    f.write(clean(req.text))

In [20]:
url = "https://www.accenture.com/"
req = requests.get(url, headers=headers)
# imgkit.from_url(url, os.path.join("webpages", str(hash(url))+".jpg"))

In [22]:
dir_name = "webpages"
url = "https://www.twitter.com/"
search_url = "https://api.apiflash.com/v1/urltoimage?access_key=37bc0ab0376f4a33b4bac2e7a47776c4&height=1920&url={}".format(url)
r=requests.get(search_url, stream=True)
r.raw.decode_content = True # handle spurious Content-Encoding
img = Image.open(r.raw)
img_file = os.path.join(dir_name, str(hash(url))+".jpg")
img.save(img_file)